# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [0]:
import pandas as pd
import numpy as np
import folium


In [17]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
fatal: destination path 'DataScienceSS20' already exists and is not an empty directory.


In [0]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [19]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [0]:
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=6/7)

In [0]:
#smaller subset of training data
X_modelOpt, _, y_modelOpt, _ = sklearn.model_selection.train_test_split(X_train, y_train, train_size=0.2)

In [0]:
#Linear Regression
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression().fit(X_modelOpt, y_modelOpt)

In [0]:
from skopt import BayesSearchCV
import numpy as np

In [16]:
%%time
#Random Forest regression
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(random_state=0, n_jobs=4)

rf_opt = BayesSearchCV(
     rf_reg,
     {
         'n_estimators': np.arange(10,100),
         'max_depth': np.arange(2,30)
     },
     n_iter=32,
     n_jobs = 4,
     random_state=0
 )

rf_opt.fit(X_modelOpt, y_modelOpt)

print(rf_opt.best_params_)
print(rf_opt.cv_results_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = yo

OrderedDict([('max_depth', 23), ('n_estimators', 99)])
defaultdict(<class 'list'>, {'split0_test_score': [0.7236731622442991, 0.7173673523639572, 0.7171033873676415, 0.6938801472482039, 0.712286207872342, 0.7248856287426347, 0.7254908280351918, 0.7215417360728925, 0.6523648306876426, 0.7218777945575932, 0.7066290223371969, 0.7234024532824154, 0.7244437546367442, 0.7257835249582426, 0.7242536560746058, 0.724819708037777, 0.7167788223652999, 0.7216097174187501, 0.7259228391880881, 0.7257681526832145, 0.7169324471981378, 0.7236579622092443, 0.6776419643923728, 0.72561880295193, 0.7253312388572346, 0.7244270454180002, 0.7244349580938316, 0.7247713722633939, 0.7249319892167639, 0.7255411578764696, 0.7259650594137472, 0.7236477458710475], 'split1_test_score': [0.80559883084211, 0.8126666715401827, 0.7916401404431915, 0.8048782055592217, 0.8069599053493561, 0.8103193026823645, 0.804931223986058, 0.8029138037600902, 0.7646490028035826, 0.8001126978942308, 0.812668179380404, 0.806888126182409, 

In [0]:
%%time
#SVM regression
from sklearn.svm import SVR

svm_reg = SVR(kernel='rbf')

svm_opt = BayesSearchCV(
     svm_reg,
     {
        'C': np.arange(1,10),
        'gamma': np.arange(0.0001,1,0.0001)
     },
     n_iter=32,
     n_jobs=-1,
     random_state=0
 )

svm_opt.fit(X_modelOpt, y_modelOpt)

print(svm_opt.best_params_)
print(svm_opt.cv_results_)

In [0]:
SMTPserver = 'mx.freenet.de'
sender =     'fabio.kraemer@freenet.de'
destination = ['fabio.kraemer@freenet.de']

USERNAME = "fabio.kraemer@freenet.de"
PASSWORD = "fTKDDez5pOD5Ju9RQzGt"

# typical values for text_subtype are plain, html, xml
text_subtype = 'plain'


content="""\
Test message
"""

subject="Job AutoSk_Learn_regression_NY_Taxy hat terminiert."

import sys
import os
import re

from smtplib import SMTP_SSL as SMTP       # this invokes the secure SMTP protocol (port 465, uses SSL)
# from smtplib import SMTP                  # use this for standard SMTP protocol   (port 25, no encryption)

# old version
# from email.MIMEText import MIMEText
from email.mime.text import MIMEText

try:
    msg = MIMEText(content, text_subtype)
    msg['Subject']=       subject
    msg['From']   = sender # some SMTP servers will do this automatically, not all

    conn = SMTP(SMTPserver)
    conn.set_debuglevel(True)
    conn.login(USERNAME, PASSWORD)
    try:
        conn.sendmail(sender, destination, msg.as_string())
    finally:
        conn.quit()

except:
    sys.exit( "mail failed; %s" % "CUSTOM_ERROR" ) # give an error message